In [0]:

CREATE STREAMING LIVE TABLE bronze_orders
TBLPROPERTIES ("quality" = "bronze")
COMMENT "Raw data loaded from volume using Auto Loader"
AS SELECT * FROM cloud_files(
  "/Volumes/dev_catalog/default/raw_volume/new_volume/",
  "csv", 
   map(
    "inferSchema", "true",
    "header", "true",
    "cloudFiles.schemaLocation", "/Volumes/dev_catalog/default/raw_volume/schema_metadata/"
  )
);

In [0]:
CREATE LIVE TABLE silver_orders
TBLPROPERTIES ("quality" = "silver")
COMMENT "Cleaned and validated data with enrichment"
AS
SELECT
  order_id,
  customer_id,
  product_id,
  product_name,
  category,
  brand,
  order_status,
  payment_method,
  location,
  CAST(quantity AS INT) AS quantity,
  gender,
  age,
  CAST(price AS DOUBLE) AS price,
  CAST(order_time AS TIMESTAMP) AS order_time,
  CAST(quantity AS INT) * CAST(price AS DOUBLE) AS total_sales
FROM LIVE.bronze_orders
WHERE 
  quantity > 0
  AND price IS NOT NULL
  AND customer_id IS NOT NULL
  AND product_id IS NOT NULL
  AND order_status IN ("placed", "shipped", "delivered", "cancelled");


In [0]:
%sql
CREATE LIVE TABLE gold_order_details
TBLPROPERTIES ("quality" = "gold")
COMMENT "Final curated order details excluding order_id and product_id"
AS
SELECT
  customer_id,
  product_name,
  category,
  brand,
  order_status,
  payment_method,
  location,
  gender,
  age,
  quantity,
  price,
  order_time,
  total_sales
FROM LIVE.silver_orders;


In [0]:
select * from dev_catalog.default.gold_order_details